In [163]:
%matplotlib inline

from IPython.core.interactiveshell import InteractiveShell
from datetime import datetime

import os

import glob
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from pandas.io.json import json_normalize


import facebook
import urllib.request
import json
import codecs

InteractiveShell.ast_node_interactivity = "all"
matplotlib.rcParams['svg.fonttype'] = 'none'

In [166]:
token = 'EAACEdEose0cBAPpHRafuE6KtShbjId3E2aEwjIfPq1WigQlHASCZA5hBhbiED9FQm6K2YdQa1cSR1OL8Pvu6DlNNnvYLJj2wBqCK1bQQQoKeO9BrCKTtaJY6TRxLp5rTc2ubMpwBi4wnKLZBETkkJtPJAvZBuVGHfpGcHZC5Y4rS4pMzqQ5ZCUevu8Sf3P7UZD'
graph = facebook.GraphAPI(access_token=token, version='2.2')

In [206]:
## load google analyctics data

ga_stats = 'ga_stats_january.csv'

stats = pd.read_csv(ga_stats)

stats['url'] = stats['Seite'].str.split('/')

for i in range(0, len(stats['url'])):   
    try:
        stats.loc[i, 'article_id'] = stats['url'][i][4]
    except:
        stats.loc[i, 'article_id'] = ''
        
stats['article_id'] = pd.to_numeric(stats['article_id'], errors='coerce')
stats['Seitenaufrufe'] = pd.to_numeric(stats['Seitenaufrufe'], errors='coerce')

stats = stats.dropna()

stats

,Seite,Seitenaufrufe,Einzelne Seitenaufrufe,Durchschn Besuchszeit auf Seite,Einstiege,Absprungrate,% Ausstiege,Seitenwert,url,article_id
0,/de/2016_53/leben/738738,11166,10070,00:04:50,8157,"87,48 %","77,49 %","0,00 CHF","[, de, 2016_53, leben, 738738]",738738.0
1,/de/2017_3/basel/740194,10377,9689,00:04:57,8512,"90,47 %","83,39 %","0,00 CHF","[, de, 2017_3, basel, 740194]",740194.0
2,/de/2017_4/schweiz/740748,8079,7660,00:02:32,7417,"85,16 %","82,88 %","0,00 CHF","[, de, 2017_4, schweiz, 740748]",740748.0
3,/de/2017_1/kultur/739630,7933,7509,00:03:24,6932,"92,25 %","87,32 %","0,00 CHF","[, de, 2017_1, kultur, 739630]",739630.0
8,/de/2017_1/basel/738891,7095,6575,00:02:58,5332,"85,60 %","74,18 %","0,00 CHF","[, de, 2017_1, basel, 738891]",738891.0
9,/de/2017_4/basel/740763,7036,6628,00:03:04,4867,"88,82 %","72,75 %","0,00 CHF","[, de, 2017_4, basel, 740763]",740763.0
11,/de/2017_3/schweiz/740240,6549,6118,00:04:13,5430,"90,99 %","83,37 %","0,00 CHF","[, de, 2017_3, schweiz, 740240]",740240.0
12,/de/2017_3/basel/740339,6426,5991,00:04:13,4344,"86,49 %","72,27 %","0,00 CHF","[, de, 2017_3, basel, 740339]",740339.0
13,/de/2017_2/basel/739636,5856,5223,00:04:19,2280,"73,11 %","52,72 %","0,00 CHF","[, de, 2017_2, basel, 739636]",739636.0
14,/de/2017_2/leben/739595,5672,5278,00:04:29,4447,"91,45 %","81,95 %","0,00 CHF","[, de, 2017_2, leben, 739595]",739595.0


In [207]:
stats['time'] = pd.DatetimeIndex(stats['Durchschn Besuchszeit auf Seite'])
stats['readtime'] = stats['time'].dt.minute * 60 + stats['time'].dt.second

In [208]:
stats_readtime = stats.groupby(by=['article_id']).readtime.mean()
stats_readtime = stats_readtime.to_frame(name = 'readtime')
stats_readtime = stats_readtime.reset_index()

stats_readtime.head()

,article_id,readtime
0,1.0,79.0
1,3915.0,124.0
2,3917.0,66.0
3,3918.0,125.0
4,3919.0,71.0


In [209]:
stats_pageviews = stats.groupby(by=['article_id']).Seitenaufrufe.sum()
stats_pageviews = stats_pageviews.to_frame(name = 'Seitenaufrufe')
stats_pageviews = stats_pageviews.reset_index()

stats_pageviews.head()

,article_id,Seitenaufrufe
0,1.0,81
1,3915.0,81
2,3917.0,367
3,3918.0,391
4,3919.0,58


In [211]:
stats = stats_pageviews.merge(stats_readtime, left_on='article_id', right_on='article_id')

In [212]:
stats.head()

,article_id,Seitenaufrufe,readtime
0,1.0,81,79.0
1,3915.0,81,124.0
2,3917.0,367,66.0
3,3918.0,391,125.0
4,3919.0,58,71.0


In [140]:
def parse_articles(id):
    meta = pd.DataFrame(columns=['type', 'article_id'])
    url = 'http://www.tageswoche.ch/content-api/articles/'
    article_name = url + str(id)
    #print(id)
    try:
        df = pd.read_json(article_name, lines=True)
        meta['type'] = df['type']
        meta['published'] = df['published']
        meta['body'] = df['fields'][0]['body']
        meta['title'] = df['title']
        meta['url'] = df['url']
        meta['comments'] = df['comments_count']
    except:
        pass
    meta['article_id'] = id
    return meta

articles_meta = pd.concat([parse_articles(id) for id in stats['article_id']])

In [141]:
articles_meta['published'] = articles_meta['published'].str.extract('(\d\d\d\d-\d\d-\d\d)')

articles_meta = articles_meta[articles_meta.published != '0001-11-30']

articles_meta['published'] = pd.to_datetime(articles_meta['published'], format='%Y-%m-%d')

articles_meta.head()

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  if __name__ == '__main__':


,article_id,body,comments,published,title,type,url
0,738738.0,"<p dir=""ltr"">Bei vielen Frauen geht Selbstbefr...",10.0,2017-01-02,Lust auf Gefühle? Probieren Sie etwas Neues aus,news,http://www.tageswoche.ch/de/2016_53/leben/7387...
0,740194.0,<p>Allein im Gesundheitssektor werden in der S...,25.0,2017-01-18,Deutsche Ärzte meiden die Schweiz – nur Basel ...,news,http://www.tageswoche.ch/de/2017_3/basel/74019...
0,740748.0,NaN,NaN,2017-01-24,NaN,newswire,NaN
0,739630.0,<p>Weiss wie frisch gefallener Schnee liegt di...,0.0,2017-01-21,"Ein Bild, das die Welt um ihren Schlaf brachte",news,http://www.tageswoche.ch/de/2017_1/kultur/7396...
0,738891.0,<p>Andreas Tännler kommt derzeit kaum zur Ruhe...,13.0,2017-01-02,«Die Solidarität ist riesig»,news,http://www.tageswoche.ch/de/2017_1/basel/73889...


In [142]:
articles_meta['body_stripped'] = articles_meta['body'].str.replace("<[^<]+?>", "")
articles_meta['body_stripped'] = articles_meta['body_stripped'].str.replace("\r\n", "")

articles_meta.head()

,article_id,body,comments,published,title,type,url,body_stripped
0,738738.0,"<p dir=""ltr"">Bei vielen Frauen geht Selbstbefr...",10.0,2017-01-02,Lust auf Gefühle? Probieren Sie etwas Neues aus,news,http://www.tageswoche.ch/de/2016_53/leben/7387...,Bei vielen Frauen geht Selbstbefriedigung so: ...
0,740194.0,<p>Allein im Gesundheitssektor werden in der S...,25.0,2017-01-18,Deutsche Ärzte meiden die Schweiz – nur Basel ...,news,http://www.tageswoche.ch/de/2017_3/basel/74019...,Allein im Gesundheitssektor werden in der Schw...
0,740748.0,NaN,NaN,2017-01-24,NaN,newswire,NaN,NaN
0,739630.0,<p>Weiss wie frisch gefallener Schnee liegt di...,0.0,2017-01-21,"Ein Bild, das die Welt um ihren Schlaf brachte",news,http://www.tageswoche.ch/de/2017_1/kultur/7396...,Weiss wie frisch gefallener Schnee liegt die F...
0,738891.0,<p>Andreas Tännler kommt derzeit kaum zur Ruhe...,13.0,2017-01-02,«Die Solidarität ist riesig»,news,http://www.tageswoche.ch/de/2017_1/basel/73889...,Andreas Tännler kommt derzeit kaum zur Ruhe. S...


In [143]:
articles_meta['length'] = articles_meta['body_stripped'].str.len()

In [144]:
articles_meta_subset = articles_meta[['article_id', 'comments', 'published', 'title', 'url', 'length', 'type']]

In [149]:
articles_meta_january = articles_meta_subset.set_index(['published'])

articles_meta_january = articles_meta_january.loc['2017-01-01':'2017-01-31']

articles_meta_january = articles_meta_january.reset_index()

In [150]:
articles_meta_january['type'] = articles_meta_january['type'].astype('category')

articles_meta_january_news = articles_meta_january[(articles_meta_january.type == 'news')]

articles_meta_january_news.head()

,published,article_id,comments,title,url,length,type
0,2017-01-02,738738.0,10.0,Lust auf Gefühle? Probieren Sie etwas Neues aus,http://www.tageswoche.ch/de/2016_53/leben/7387...,6277.0,news
1,2017-01-18,740194.0,25.0,Deutsche Ärzte meiden die Schweiz – nur Basel ...,http://www.tageswoche.ch/de/2017_3/basel/74019...,6114.0,news
3,2017-01-21,739630.0,0.0,"Ein Bild, das die Welt um ihren Schlaf brachte",http://www.tageswoche.ch/de/2017_1/kultur/7396...,4056.0,news
4,2017-01-02,738891.0,13.0,«Die Solidarität ist riesig»,http://www.tageswoche.ch/de/2017_1/basel/73889...,2343.0,news
5,2017-01-26,740763.0,35.0,Was stimmt mit dieser Strasse nicht?,http://www.tageswoche.ch/de/2017_4/basel/74076...,2974.0,news


In [151]:
articles_meta_january_news['article_id'].nunique()

228

In [231]:
# merge google stats and article meta data

data_january_subset = articles_meta_january_news.merge(stats, left_on='article_id', right_on='article_id')

data_january_subset

,published,article_id,comments,title,url,length,type,Seitenaufrufe,readtime
0,2017-01-02,738738.0,10.0,Lust auf Gefühle? Probieren Sie etwas Neues aus,http://www.tageswoche.ch/de/2016_53/leben/7387...,6277.0,news,11737,272.333333
1,2017-01-02,738738.0,10.0,Lust auf Gefühle? Probieren Sie etwas Neues aus,http://www.tageswoche.ch/de/2016_53/leben/7387...,6277.0,news,11737,272.333333
2,2017-01-02,738738.0,10.0,Lust auf Gefühle? Probieren Sie etwas Neues aus,http://www.tageswoche.ch/de/2016_53/leben/7387...,6277.0,news,11737,272.333333
3,2017-01-18,740194.0,25.0,Deutsche Ärzte meiden die Schweiz – nur Basel ...,http://www.tageswoche.ch/de/2017_3/basel/74019...,6114.0,news,10575,248.000000
4,2017-01-18,740194.0,25.0,Deutsche Ärzte meiden die Schweiz – nur Basel ...,http://www.tageswoche.ch/de/2017_3/basel/74019...,6114.0,news,10575,248.000000
5,2017-01-21,739630.0,0.0,"Ein Bild, das die Welt um ihren Schlaf brachte",http://www.tageswoche.ch/de/2017_1/kultur/7396...,4056.0,news,8003,356.333333
6,2017-01-21,739630.0,0.0,"Ein Bild, das die Welt um ihren Schlaf brachte",http://www.tageswoche.ch/de/2017_1/kultur/7396...,4056.0,news,8003,356.333333
7,2017-01-21,739630.0,0.0,"Ein Bild, das die Welt um ihren Schlaf brachte",http://www.tageswoche.ch/de/2017_1/kultur/7396...,4056.0,news,8003,356.333333
8,2017-01-02,738891.0,13.0,«Die Solidarität ist riesig»,http://www.tageswoche.ch/de/2017_1/basel/73889...,2343.0,news,7632,97.250000
9,2017-01-02,738891.0,13.0,«Die Solidarität ist riesig»,http://www.tageswoche.ch/de/2017_1/basel/73889...,2343.0,news,7632,97.250000


In [214]:
comments_mean = data_january_subset.comments.mean()

In [215]:
readtime_mean = data_january_subset.readtime.mean()

In [216]:
pageviews_mean = data_january_subset.Seitenaufrufe.mean()

In [167]:
def get_facebook_shares(url):
    shares = pd.DataFrame(columns=['url', 'share_count'])
    shares_new = ['', '']  
    shares.loc[len(shares)] = shares_new
    try:
        post = graph.get_object(id=url)
        shares['share_count'] = post['share']['share_count']
        shares['url'] = url
    except:
        pass
        
    return shares

share_stats = pd.concat([get_facebook_shares(url) for url in data_january_subset['url']])

share_stats = share_stats.reset_index()
share_stats = share_stats[[1,2]]

share_stats.head()

,url,share_count
0,http://www.tageswoche.ch/de/2016_53/leben/7387...,23
1,http://www.tageswoche.ch/de/2016_53/leben/7387...,23
2,http://www.tageswoche.ch/de/2016_53/leben/7387...,23
3,http://www.tageswoche.ch/de/2017_3/basel/74019...,48
4,http://www.tageswoche.ch/de/2017_3/basel/74019...,48


In [174]:
shares_mean = share_stats.share_count.mean()

In [185]:
data_subset = data_january_subset.merge(share_stats, left_on='url', right_on='url')

In [220]:
data_subset = data_subset.drop_duplicates()

In [195]:
data_subset['length'].mean()

4600.4298245614036

In [200]:
readtime_mean = data_subset['readtime'].mean()

In [221]:
data_subset['kennzahl_shares'] = data_subset['share_count'] / shares_mean
data_subset['kennzahl_comments'] = data_subset['comments'] / comments_mean
data_subset['kennzahl_readtime'] = data_subset['readtime'] / readtime_mean
data_subset['kennzahl_pageviews'] = data_subset['Seitenaufrufe'] / pageviews_mean


data_subset.head()

,published,article_id,comments,title,url,length,type,Seitenaufrufe,readtime,share_count,kennzahl_shares,kennzahl_comments,kennzahl_readtime,kennzahl_pageviews,erfolgsindex
0,2017-01-02,738738.0,10.0,Lust auf Gefühle? Probieren Sie etwas Neues aus,http://www.tageswoche.ch/de/2016_53/leben/7387...,6277.0,news,11737,272.333333,23,0.344107,1.504339,1.459213,5.737830,2.573210
9,2017-01-18,740194.0,25.0,Deutsche Ärzte meiden die Schweiz – nur Basel ...,http://www.tageswoche.ch/de/2017_3/basel/74019...,6114.0,news,10575,248.000000,48,0.718136,3.760849,1.328830,5.169766,2.824217
13,2017-01-21,739630.0,0.0,"Ein Bild, das die Welt um ihren Schlaf brachte",http://www.tageswoche.ch/de/2017_1/kultur/7396...,4056.0,news,8003,356.333333,6,0.089767,0.000000,1.909300,3.912401,1.687154
22,2017-01-02,738891.0,13.0,«Die Solidarität ist riesig»,http://www.tageswoche.ch/de/2017_1/basel/73889...,2343.0,news,7632,97.250000,241,3.605639,1.955641,0.521084,3.731031,2.939343
38,2017-01-26,740763.0,35.0,Was stimmt mit dieser Strasse nicht?,http://www.tageswoche.ch/de/2017_4/basel/74076...,2974.0,news,7303,144.000000,38,0.568524,5.265188,0.771579,3.570194,2.354066


In [228]:
data_subset['erfolgsindex'] = ((data_subset['kennzahl_shares'] * 30) + (data_subset['kennzahl_pageviews'] * 40) + (data_subset['kennzahl_comments'] * 15) + (data_subset['kennzahl_readtime'] * 15)) / 100 

In [229]:
data_subset.sort_values('erfolgsindex', ascending=False)

,published,article_id,comments,title,url,length,type,Seitenaufrufe,readtime,share_count,kennzahl_shares,kennzahl_comments,kennzahl_readtime,kennzahl_pageviews,erfolgsindex
223,2017-01-15,739248.0,26.0,Warum die Schweizer Steuerreform Entwicklungsl...,http://www.tageswoche.ch/de/2017_1/schweiz/739...,5037.0,news,3859,246.333333,822,12.298072,3.911283,1.319900,1.886537,5.228714
58,2017-01-23,740339.0,29.0,Diese Firmen profitieren von der Steuerreform ...,http://www.tageswoche.ch/de/2017_3/basel/74033...,6860.0,news,7080,207.800000,550,8.228637,4.362584,1.113431,3.461177,4.674464
42,2017-01-22,740240.0,21.0,Schändlich verschwendetes Potenzial,http://www.tageswoche.ch/de/2017_3/schweiz/740...,6348.0,news,7584,176.750000,478,7.151434,3.159113,0.947059,3.707566,4.244382
9,2017-01-18,740194.0,25.0,Deutsche Ärzte meiden die Schweiz – nur Basel ...,http://www.tageswoche.ch/de/2017_3/basel/74019...,6114.0,news,10575,248.000000,48,0.718136,3.760849,1.328830,5.169766,3.046799
22,2017-01-02,738891.0,13.0,«Die Solidarität ist riesig»,http://www.tageswoche.ch/de/2017_1/basel/73889...,2343.0,news,7632,97.250000,241,3.605639,1.955641,0.521084,3.731031,2.945613
0,2017-01-02,738738.0,10.0,Lust auf Gefühle? Probieren Sie etwas Neues aus,http://www.tageswoche.ch/de/2016_53/leben/7387...,6277.0,news,11737,272.333333,23,0.344107,1.504339,1.459213,5.737830,2.842897
441,2017-01-26,740766.0,0.0,Rebell auf Rädern,http://www.tageswoche.ch/de/2017_4/basel/74076...,6563.0,news,2814,233.666667,426,6.373454,0.000000,1.252029,1.375671,2.650109
38,2017-01-26,740763.0,35.0,Was stimmt mit dieser Strasse nicht?,http://www.tageswoche.ch/de/2017_4/basel/74076...,2974.0,news,7303,144.000000,38,0.568524,5.265188,0.771579,3.570194,2.504150
378,2017-01-16,739562.0,20.0,Bloss keine Verpackung – Abfallvermeiderin Car...,http://www.tageswoche.ch/de/2017_2/basel/73956...,5568.0,news,2749,289.750000,263,3.934785,3.008679,1.552534,1.343895,2.402175
258,2017-01-05,738972.0,19.0,Subventionen für alte Atomkraftwerke: Blochers...,http://www.tageswoche.ch/de/2017_1/basel/73897...,26244.0,news,3083,403.666667,222,3.321377,2.858245,2.162921,1.507176,2.352459


In [230]:
data_subset.to_csv('erfolgsindex_test.csv')